# MAXI

In [916]:
import pandas as pd

### Ventas y Volumen

In [917]:
cols = ['Año', 'Mes', 'Direccion', 'Punto Operacional', 'Sector', 'Seccion', 'Grupo de Familia', 'Ventas c/impuesto', 'Venta en Unidades']
df_ventas_vol = pd.read_csv('data/ventas y volumen maxi 28 dias moviles sep-oct.csv', encoding='utf-16', header=1, usecols=cols, decimal=',')
df_ventas_vol.columns = df_ventas_vol.columns.str.lower().str.replace(' ','_')

In [918]:
df_ventas_vol.rename(columns={
    'mes':'fecha',
    'ventas_c/impuesto':'vct',
    'venta_en_unidades':'vol',
}, inplace=True)

In [919]:
df_ventas_vol['mes'] = df_ventas_vol['fecha'].str.split(' ').str[0]
df_ventas_vol['numero_operacional'] = df_ventas_vol['punto_operacional'].str.split(' ').str[0].astype(int)

### Separacion Ventas y Volumen

In [920]:
df_ventas = df_ventas_vol[['año', 'fecha', 'direccion', 'punto_operacional', 'sector', 'seccion', 'grupo_de_familia', 'vct', 'mes', 'numero_operacional']]
df_volumen = df_ventas_vol[['año', 'fecha', 'direccion', 'punto_operacional', 'sector', 'seccion', 'grupo_de_familia', 'vol', 'mes', 'numero_operacional']]
df_ventas.rename(columns={
    'vct':'valores'
}, inplace=True)
df_volumen.rename(columns={
    'vol':'valores'
}, inplace=True)

C:\Users\juan_mera\AppData\Local\Temp\ipykernel_12020\3609712812.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ventas.rename(columns={
C:\Users\juan_mera\AppData\Local\Temp\ipykernel_12020\3609712812.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_volumen.rename(columns={


In [921]:
df_ventas['categoria'] = 'vct'
df_volumen['categoria'] = 'vol'

In [922]:
df_ventas['valores'] = pd.to_numeric(df_ventas['valores'].str.replace('.', '').str.replace(',','.'))
df_volumen['valores'] = pd.to_numeric(df_volumen['valores'].str.replace('.', '').str.replace(',','.'))

### Le quito los envases al volumen

In [923]:
df_volumen = df_volumen[~df_volumen['grupo_de_familia'].isin(['ENVASES BEBIDAS', 'ENVASES PAGADOS'])]

In [924]:
df_ventas_tienda = df_ventas.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()
df_volumen_tienda = df_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()

### Padron

In [925]:
cols = ['N°', 'NOMBRE', 'Fecha apertura', 'BANDERA', 'ORGANIZACIÓN ', 'PROVINCIA', 'FIN DE CIERRE', 'ENE.2', 'FEB.2', 'MAR.2', 'ABR.2', 'MAY.2', 'JUN.2', 'JUL.2', 'AGO.2', 'SEP.2', 'OCT.2', 'NOV.2', 'DIC.2']
padron = pd.read_excel('data/padron.xlsx', header=17, usecols=cols)
padron.columns = padron.columns.str.lower().str.strip().str.replace(' ', '_').str.replace('.2', '')
padron.rename(columns={
    'n°':'numero_operacional',
}, inplace=True)

In [926]:
#Elimino las filas que tengan na en su numero, nombre y mes comparable
padron = padron.dropna(subset=['numero_operacional', 'nombre', 'sep'], how='any')

In [927]:
padron['numero_operacional'] = padron['numero_operacional'].astype(int)

### Debitos total Tienda

In [928]:
df_debitos_tienda = pd.read_csv('data/debitos total tienda maxi 28 dias moviles sep-oct.csv', encoding='utf-16', header=1, decimal=',')
df_debitos_tienda.columns = df_debitos_tienda.columns.str.lower().str.replace(' ', '_')
df_debitos_tienda.rename(columns={
    'cant._tickets_por_local':'valores',
    'mes':'fecha',
}, inplace=True)
df_debitos_tienda['valores'] = pd.to_numeric(df_debitos_tienda['valores'].str.replace('.', '').str.replace(',','.'))
df_debitos_tienda['mes'] = df_debitos_tienda['fecha'].str.split(' ').str[0]
df_debitos_tienda['numero_operacional'] = df_debitos_tienda['punto_operacional'].str.split(' ').str[0].astype(int)
df_debitos_tienda = df_debitos_tienda.drop(columns=['indicadores', 'fecha'])
df_debitos_tienda['categoria'] = 'deb'
df_debitos_tienda = df_debitos_tienda[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'valores']]

### Concateno todo nivel tienda y Join con el Padron

In [929]:
df_tienda = pd.concat([df_ventas_tienda, df_debitos_tienda, df_volumen_tienda])
df_tienda['numero_operacional'] = df_tienda['numero_operacional'].astype(int)

In [930]:
df_tienda_join = pd.merge(df_tienda, padron, how='left', on='numero_operacional')
df_tienda_join = df_tienda_join[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'provincia', 'fecha_apertura', 'fin_de_cierre','categoria', 'sep', 'valores']]
df_tienda_comparable = df_tienda_join[df_tienda_join['sep'] == 'SC']

In [931]:
df_tienda_comparable[df_tienda_comparable['categoria'] == 'deb']['valores'].sum()

np.float64(1359871.0)

### Genero un df por Superficie comparable, pivoteo la info y calculo las porgresiones totales por tienda por categoria

In [932]:
df_tienda_comparable = df_tienda_comparable.pivot_table(values='valores', index=['numero_operacional', 'punto_operacional', 'categoria'], columns='año', aggfunc='sum').reset_index()
df_tienda_comparable['progresion'] = round((df_tienda_comparable[2025] / df_tienda_comparable[2024]) - 1, 3)
df_tienda_comparable = df_tienda_comparable.sort_values(by='progresion', ascending=False)

### Debitos por Sector

In [933]:
df_debitos_sector = pd.read_csv('data/debitos por sector maxi 28 dias movil sep-oct.csv', encoding='utf-16', header=1, decimal=',')

In [934]:
df_debitos_sector.columns = df_debitos_sector.columns.str.strip().str.lower().str.replace(' ', '_')
df_debitos_sector = df_debitos_sector.rename(columns={
    'cantidad_de_tickets':'valores',
    'mes':'fecha'
})
df_debitos_sector = df_debitos_sector.drop(columns=['indicadores'])
df_debitos_sector['mes'] = df_debitos_sector['fecha'].str.split(' ').str[0]
df_debitos_sector['numero_operacional'] = df_debitos_sector['punto_operacional'].str.split(' ').str[0]
df_debitos_sector['categoria'] = 'deb'
df_debitos_sector['valores'] = pd.to_numeric(df_debitos_sector['valores'].str.replace('.', '').str.replace(',','.'))

### Agrupo las tres categorias por Sector

In [935]:
df_ventas_sector = df_ventas.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'sector'])['valores'].sum().reset_index()
df_volumen_sector = df_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'sector'])['valores'].sum().reset_index()
df_debitos_sector = df_debitos_sector[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'sector', 'valores']]

In [936]:
df_sector = pd.concat([df_ventas_sector, df_volumen_sector, df_debitos_sector])
df_sector['numero_operacional'] = df_sector['numero_operacional'].astype(int) 
df_sector_join = pd.merge(df_sector, padron, on='numero_operacional', how='left')
df_sector_join = df_sector_join[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'provincia', 'fecha_apertura', 'fin_de_cierre', 'categoria', 'sector', 'sep', 'valores']]

In [937]:
df_sector_comparable = df_sector_join[df_sector_join['sep'] == 'SC']

In [938]:
df_sector_comparable = df_sector_comparable.pivot_table(values='valores', index=['numero_operacional', 'punto_operacional', 'categoria', 'sector'], columns='año', aggfunc='sum').reset_index()
df_sector_comparable['progresion'] = round((df_sector_comparable[2025] / df_sector_comparable[2024]) - 1, 3)

In [939]:
df_progresiones_categoria_sectores = df_sector_comparable.pivot_table(values='progresion', index=['numero_operacional', 'punto_operacional', 'categoria'], columns='sector', aggfunc='sum').reset_index()

In [940]:
df_tienda_comparable = df_tienda_comparable.rename(columns={'progresion':'total_tienda'})

In [941]:
df_progresiones_join_sector_tienda = pd.merge(df_progresiones_categoria_sectores, df_tienda_comparable[['numero_operacional', 'categoria', 'total_tienda']], on=['numero_operacional', 'categoria'], how='left')

In [942]:
df_progresiones_join_sector_tienda = df_progresiones_join_sector_tienda.fillna(0)

In [943]:
df_progresiones_join_sector_tienda.columns = df_progresiones_join_sector_tienda.columns.str.capitalize().str.strip().str.replace('_', ' ')

In [944]:
df_progresiones_join_sector_tienda = df_progresiones_join_sector_tienda.drop(columns=['Numero operacional'])
df_progresiones_join_sector_tienda = df_progresiones_join_sector_tienda.rename(columns={'Total tienda': 'Total tienda', 'P.g.c.': 'PGC'})
df_progresiones_join_sector_tienda = df_progresiones_join_sector_tienda.sort_values(by='Total tienda', ascending=False)

In [945]:
df_progresiones_tienda_sector_total_vct = df_progresiones_join_sector_tienda[df_progresiones_join_sector_tienda['Categoria'] == 'vct'].drop(columns='Categoria')
df_progresiones_tienda_sector_total_deb = df_progresiones_join_sector_tienda[df_progresiones_join_sector_tienda['Categoria'] == 'deb'].drop(columns='Categoria')
df_progresiones_tienda_sector_total_vol = df_progresiones_join_sector_tienda[df_progresiones_join_sector_tienda['Categoria'] == 'vol'].drop(columns='Categoria')

In [946]:
top_tiendas_vct = df_progresiones_tienda_sector_total_vct.head(5)
flop_tiendas_vct = df_progresiones_tienda_sector_total_vct.tail(5)
top_tiendas_deb = df_progresiones_tienda_sector_total_deb.head(5)
flop_tiendas_deb = df_progresiones_tienda_sector_total_deb.tail(5)
top_tiendas_vol = df_progresiones_tienda_sector_total_vol.head(5)
flop_tiendas_vol = df_progresiones_tienda_sector_total_vol.tail(5)

In [947]:
df_final_consolidado_tienda = df_tienda_comparable.drop(columns=[2024, 2025])
df_final_consolidado_tienda = df_final_consolidado_tienda.rename(columns={'total_tienda':'progresion'})
df_final_consolidado_tienda['sector'] = 'Total'

In [948]:
df_final_consolidado_sector = df_sector_comparable.drop(columns=[2024, 2025])
df_final_consolidado_sector[['numero_operacional', 'punto_operacional', 'categoria', 'progresion', 'sector']]

año,numero_operacional,punto_operacional,categoria,progresion,sector
0,20,020 - Avellaneda,deb,-0.044,BAZAR
1,20,020 - Avellaneda,deb,0.019,ELECTRODOMESTICOS
2,20,020 - Avellaneda,deb,-0.169,OTROS
3,20,020 - Avellaneda,deb,-0.119,P.G.C.
4,20,020 - Avellaneda,deb,-0.072,PRODUCTOS FRESCO
...,...,...,...,...,...
559,896,896 - MAXI - SIDE STORE,deb,0.000,PRODUCTOS FRESCO
560,896,896 - MAXI - SIDE STORE,vct,NaN,P.G.C.
561,896,896 - MAXI - SIDE STORE,vct,-0.554,PRODUCTOS FRESCO
562,896,896 - MAXI - SIDE STORE,vol,NaN,P.G.C.


In [949]:
df_final_consolidado_total = pd.concat([df_final_consolidado_sector, df_final_consolidado_tienda])
df_final_consolidado_total = df_final_consolidado_total.pivot_table(values='progresion', index='punto_operacional', columns=['categoria', 'sector'], aggfunc='sum').reset_index()

### Trabajo sobre el ultimo punto "Grupo de Familia"

In [950]:
df_volumen_grupo_de_familia = df_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'sector', 'seccion', 'grupo_de_familia'])['valores'].sum().reset_index()

In [951]:
df_volumen_grupo_de_familia_join = pd.merge(df_volumen_grupo_de_familia, padron[['numero_operacional', 'sep']], on='numero_operacional', how='left')

In [952]:
df_volumen_grupo_de_familia_comparable = df_volumen_grupo_de_familia_join[df_volumen_grupo_de_familia_join['sep'] == 'SC']

In [953]:
df_volumen_grupo_de_familia_comparable['valores'].sum()

np.float64(57404534.849999994)

In [954]:
df_volumen_grupo_de_familia_comparable = df_volumen_grupo_de_familia_comparable.pivot_table(values='valores', index=['grupo_de_familia', 'seccion', 'categoria'], columns='año', aggfunc='sum').reset_index()

In [955]:
df_volumen_grupo_de_familia_comparable[2024] = df_volumen_grupo_de_familia_comparable[2024].fillna(0)

In [956]:
df_volumen_grupo_de_familia_comparable['GAP'] = df_volumen_grupo_de_familia_comparable[2025] - df_volumen_grupo_de_familia_comparable[2024]

In [957]:
df_volumen_grupo_de_familia_comparable['progresion'] = (df_volumen_grupo_de_familia_comparable[2025] / df_volumen_grupo_de_familia_comparable[2024]) - 1

In [958]:
import numpy as np

df_volumen_grupo_de_familia_comparable['progresion'] = df_volumen_grupo_de_familia_comparable['progresion'].replace(np.inf, 0)
df_volumen_grupo_de_familia_comparable['progresion'] = df_volumen_grupo_de_familia_comparable['progresion'].replace(np.nan, 0)
df_volumen_grupo_de_familia_comparable.sort_values('progresion', ascending=False)
total_2024 = df_volumen_grupo_de_familia_comparable[2024].sum()
df_volumen_grupo_de_familia_comparable['Cmg'] = df_volumen_grupo_de_familia_comparable['GAP'] / total_2024

In [959]:
df_volumen_grupo_de_familia_comparable['Cmg'] = df_volumen_grupo_de_familia_comparable['Cmg'].fillna(0)
df_volumen_grupo_de_familia_comparable = df_volumen_grupo_de_familia_comparable.sort_values('Cmg', ascending=False)
df_volumen_grupo_de_familia_comparable = df_volumen_grupo_de_familia_comparable.rename(columns={'grupo_de_familia':'Grupo de familia', 'seccion':'Seccion', 'categoria':'Categoria', 'progresion':'Progresion'})

In [960]:
df_volumen_grupo_de_familia_comparable_top = df_volumen_grupo_de_familia_comparable.head(5)
df_volumen_grupo_de_familia_comparable_flop = df_volumen_grupo_de_familia_comparable.tail(5)

In [961]:
'''from pathlib import Path

try:
    out_dir = Path("results")
    out_dir.mkdir(parents=True, exist_ok=True)
    out_file = out_dir / f"Resultados Briefing.xlsx"

    with pd.ExcelWriter(out_file) as writer:

        top_tiendas_vct.to_excel(writer, sheet_name='Top VCT', index=False)
        flop_tiendas_vct.to_excel(writer, sheet_name='Flop VCT', index=False)
        top_tiendas_deb.to_excel(writer, sheet_name='Top DEB', index=False)
        flop_tiendas_deb.to_excel(writer, sheet_name='Flop DEB', index=False)
        top_tiendas_vol.to_excel(writer, sheet_name='Top VOL', index=False)
        flop_tiendas_vol.to_excel(writer, sheet_name='Flop VOL', index=False)
        df_final_consolidado_total.to_excel(writer, sheet_name='Info Consolidada', index=True)
        df_volumen_grupo_de_familia_comparable_top.to_excel(writer, sheet_name='GF Top', index=False)
        df_volumen_grupo_de_familia_comparable_flop.to_excel(writer, sheet_name='GF Flop', index=False)
        df_volumen_grupo_de_familia_comparable.to_excel(writer, sheet_name='GF Consolidado', index=False)
          
    print(f"Archivo guardado en: {out_file}")

except Exception as e:
    print(f"No se logró guardar la información en un archivo Excel. ERROR: {e}")'''

'from pathlib import Path\n\ntry:\n    out_dir = Path("results")\n    out_dir.mkdir(parents=True, exist_ok=True)\n    out_file = out_dir / f"Resultados Briefing.xlsx"\n\n    with pd.ExcelWriter(out_file) as writer:\n\n        top_tiendas_vct.to_excel(writer, sheet_name=\'Top VCT\', index=False)\n        flop_tiendas_vct.to_excel(writer, sheet_name=\'Flop VCT\', index=False)\n        top_tiendas_deb.to_excel(writer, sheet_name=\'Top DEB\', index=False)\n        flop_tiendas_deb.to_excel(writer, sheet_name=\'Flop DEB\', index=False)\n        top_tiendas_vol.to_excel(writer, sheet_name=\'Top VOL\', index=False)\n        flop_tiendas_vol.to_excel(writer, sheet_name=\'Flop VOL\', index=False)\n        df_final_consolidado_total.to_excel(writer, sheet_name=\'Info Consolidada\', index=True)\n        df_volumen_grupo_de_familia_comparable_top.to_excel(writer, sheet_name=\'GF Top\', index=False)\n        df_volumen_grupo_de_familia_comparable_flop.to_excel(writer, sheet_name=\'GF Flop\', index=

### Graficos

In [962]:
df_debitos = pd.read_csv('data/debitos maxi 2023 - 2025.csv', encoding='utf-16', header=1, decimal=',')

In [963]:
meses_orden = {'Enero':1, 'Febrero':2, 'Marzo':3, 'Abril':4, 'Mayo':5, 'Junio':6, 'Julio':7, 'Agosto':8, 'Septiembre':9, 'Octubre':10, 'Noviembre':11, 'Diciembre':12}

In [964]:
df_debitos.columns = df_debitos.columns.str.strip().str.replace(' ', '_').str.lower()
df_debitos = df_debitos.rename(columns={'mes':'fecha', 'cant._tickets_por_local': 'valores'})
df_debitos['mes'] = df_debitos['fecha'].str.split(' ').str[0]
df_debitos['numero_operacional'] = df_debitos['punto_operacional'].str.split(' ').str[0].astype(int)
df_debitos = df_debitos.drop(columns=['indicadores'])
df_debitos['valores'] = df_debitos['valores'].str.replace('.','').str.replace(',','.').astype(int)
df_debitos['categoria'] = 'deb'

In [965]:
df_debitos

,año,fecha,punto_operacional,direccion,valores,mes,numero_operacional,categoria
0,2023,Enero del 2023,020 - Avellaneda,MAXI,40016,Enero,20,deb
1,2023,Enero del 2023,040 - Temperley,MAXI,34536,Enero,40,deb
2,2023,Enero del 2023,045 - Trelew 25 de Mayo,MAXI,30130,Enero,45,deb
3,2023,Enero del 2023,057 - Tandil Villa Italia,MAXI,36672,Enero,57,deb
4,2023,Enero del 2023,059 - Mendoza Godoy Cruz,MAXI,45809,Enero,59,deb
...,...,...,...,...,...,...,...,...
1032,2025,Octubre del 2025,706 - Maxi Lomas,MAXI,5188,Octubre,706,deb
1033,2025,Octubre del 2025,707 - Maxi Cordoba J.B.Justo,MAXI,7559,Octubre,707,deb
1034,2025,Octubre del 2025,708 - Maxi Tortuguitas,MAXI,4735,Octubre,708,deb
1035,2025,Octubre del 2025,709 - MAXI - LOMA HERMOSA,MAXI,4149,Octubre,709,deb


In [966]:
cols = ['Año', 'Mes', 'Direccion', 'Punto Operacional', 'Grupo de Familia', 'Ventas c/impuesto', 'Venta en Unidades']
df_ventas = pd.read_csv('data/ventas maxi 2023 -2025.csv', encoding='utf-16', header=1, usecols=cols)

In [967]:
df_ventas.columns = df_ventas.columns.str.lower().str.strip().str.replace(' ', '_')

df_ventas = df_ventas.rename(columns={
    'mes':'fecha',
    'ventas_c/impuesto':'ventas',
    'venta_en_unidades':'volumen'
})

df_ventas['mes'] = df_ventas['fecha'].str.split(' ').str[0]
df_ventas['numero_operacional'] = df_ventas['punto_operacional'].str.split(' ').str[0].astype(int)

df_ventas['ventas'] = df_ventas['ventas'].str.replace('.', '').str.replace(',', '.').astype(float)
df_ventas['volumen'] = df_ventas['volumen'].str.replace('.', '').str.replace(',', '.').astype(float)

In [968]:
df_volumen = df_ventas[['año', 'fecha', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'grupo_de_familia', 'volumen']]
df_ventas = df_ventas[['año', 'fecha', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'grupo_de_familia', 'ventas' ]]

In [969]:
df_ventas = df_ventas.rename(columns={
    'ventas':'valores'
})
df_ventas['categoria'] =  'vct'
df_ventas = df_ventas.groupby(['año', 'fecha', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()

In [970]:
df_volumen = df_volumen.rename(columns={
    'volumen':'valores'
})
df_volumen['categoria'] =  'vol'
df_volumen = df_volumen[~df_volumen['grupo_de_familia'].isin(['ENVASES BEBIDAS', 'ENVASES PAGADOS'])]
df_volumen = df_volumen.groupby(['año', 'fecha', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()

In [971]:
df_debitos = df_debitos[['año', 'fecha', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'valores']]

In [972]:
df_vdv = pd.concat([df_ventas, df_volumen, df_debitos])

In [973]:
df_vdv_join = pd.merge(df_vdv, padron[['numero_operacional', 'sep']], on='numero_operacional')

In [974]:
df_vdv_join['aux'] = df_vdv_join['mes'].map(meses_orden)

In [975]:
meses_orden.items()
meses_invertidos = {v:k for k, v in meses_orden.items()}
meses_invertidos

{1: 'Enero',
 2: 'Febrero',
 3: 'Marzo',
 4: 'Abril',
 5: 'Mayo',
 6: 'Junio',
 7: 'Julio',
 8: 'Agosto',
 9: 'Septiembre',
 10: 'Octubre',
 11: 'Noviembre',
 12: 'Diciembre'}

In [976]:
df_vdv_join

,año,fecha,mes,direccion,numero_operacional,punto_operacional,categoria,valores,sep,aux
0,2023,Abril del 2023,Abril,MAXI,20,020 - Avellaneda,vct,9.440494e+08,SC,4
1,2023,Abril del 2023,Abril,MAXI,40,040 - Temperley,vct,2.997863e+08,SC,4
2,2023,Abril del 2023,Abril,MAXI,45,045 - Trelew 25 de Mayo,vct,1.980232e+08,SC,4
3,2023,Abril del 2023,Abril,MAXI,57,057 - Tandil Villa Italia,vct,3.037086e+08,SC,4
4,2023,Abril del 2023,Abril,MAXI,59,059 - Mendoza Godoy Cruz,vct,7.150335e+08,SC,4
...,...,...,...,...,...,...,...,...,...,...
3108,2025,Octubre del 2025,Octubre,MAXI,706,706 - Maxi Lomas,deb,5.188000e+03,SC,10
3109,2025,Octubre del 2025,Octubre,MAXI,707,707 - Maxi Cordoba J.B.Justo,deb,7.559000e+03,SC,10
3110,2025,Octubre del 2025,Octubre,MAXI,708,708 - Maxi Tortuguitas,deb,4.735000e+03,SC,10
3111,2025,Octubre del 2025,Octubre,MAXI,709,709 - MAXI - LOMA HERMOSA,deb,4.149000e+03,SC,10


In [977]:
df_vdv_join.sort_values(by='aux', ascending=True)

,año,fecha,mes,direccion,numero_operacional,punto_operacional,categoria,valores,sep,aux
90,2023,Enero del 2023,Enero,MAXI,266,266 - Parana Don Bosco,vct,7.080923e+07,SC,1
91,2023,Enero del 2023,Enero,MAXI,278,278 - MAXI Marquez,vct,1.686451e+08,SC,1
92,2023,Enero del 2023,Enero,MAXI,294,294 - Ituzaingo Blas Parera,vct,2.159977e+08,SC,1
93,2023,Enero del 2023,Enero,MAXI,701,701 - Ruta III,vct,7.070009e+08,SC,1
94,2023,Enero del 2023,Enero,MAXI,702,702 - Claypole,vct,7.535027e+08,SC,1
...,...,...,...,...,...,...,...,...,...,...
77,2023,Diciembre del 2023,Diciembre,MAXI,709,709 - MAXI - LOMA HERMOSA,vct,7.033901e+08,SC,12
78,2023,Diciembre del 2023,Diciembre,MAXI,896,896 - MAXI - SIDE STORE,vct,1.163335e+08,SC,12
2366,2023,Diciembre del 2023,Diciembre,MAXI,503,503 - Maxi San Vicente,deb,2.748600e+04,SC,12
2367,2023,Diciembre del 2023,Diciembre,MAXI,504,504 - Maxi Cacheuta Cba,deb,2.684500e+04,SC,12


In [978]:
df_vdv_join['fecha_completa'] = pd.to_datetime(
    '01/' + df_vdv_join['aux'].astype(str) + '/' + df_vdv_join['año'].astype(str), format='%d/%m/%Y')

In [979]:
mes_comparable = 'Septiembre'
fecha_tope = pd.to_datetime('01/'+ str(meses_orden[mes_comparable]) + '/' + '2025', format='%d/%m/%Y')

In [980]:
df_vdv_join = df_vdv_join[df_vdv_join['fecha_completa'] <= fecha_tope]

In [981]:
df_vdv_join = df_vdv_join.pivot_table(values='valores', columns='año', index=['mes', 'numero_operacional', 'punto_operacional', 'categoria', 'sep', 'aux'], aggfunc='sum').reset_index()

In [982]:
df_vdv_join['progresion 2024'] = round((df_vdv_join[2024] / df_vdv_join[2023]) - 1, 3)
df_vdv_join['progresion 2025'] = round((df_vdv_join[2025] / df_vdv_join[2024]) - 1, 3)

In [983]:
df_vdv_join_tienda = df_vdv_join.groupby(['mes', 'numero_operacional', 'punto_operacional', 'categoria', 'sep', 'aux'])[[2023, 2024, 2025, 'progresion 2024', 'progresion 2025']].sum().reset_index()

In [984]:
df_vdv_join_tienda = df_vdv_join_tienda.pivot_table(values=[2024, 2025, 'progresion 2024', 'progresion 2025'], columns='categoria', index=['mes', 'numero_operacional', 'punto_operacional', 'sep', 'aux'], aggfunc='sum').reset_index().sort_values(by=('aux',''))

In [985]:
df_vdv_join_comparable = df_vdv_join[df_vdv_join['sep'] == 'SC']
df_vdv_join_total = df_vdv_join

In [986]:
info_comparable = df_vdv_join_comparable.groupby(['mes', 'categoria', 'aux'])[['progresion 2024', 'progresion 2025']].sum().reset_index().pivot_table(values=['progresion 2024', 'progresion 2025'], index=['mes', 'aux'], columns='categoria').reset_index().sort_values(('aux',''))

In [987]:
info_comparable

año               mes aux progresion 2024                progresion 2025  \
categoria                             deb     vct    vol             deb   
3               Enero   1           1.381  60.487  0.355          -6.497   
4             Febrero   2          -0.154  70.576  0.524          -6.019   
7               Marzo   3          -2.231  61.121 -1.550          -5.124   
0               Abril   4          -4.103  51.952 -3.600          -4.165   
8                Mayo   5          -3.031  49.917 -3.630          -5.305   
6               Junio   6          -3.594  46.074 -3.974          -6.144   
5               Julio   7          -4.056  41.593 -5.538          -5.370   
1              Agosto   8          -5.862  36.663 -6.107          -4.423   
11         Septiembre   9          -6.925  46.838 -5.847          -5.486   
10            Octubre  10          -6.188  37.826 -5.516           0.000   
9           Noviembre  11           8.905  62.199  0.320           0.000   
2           Diciembre  12          -8.680  23.551 -7.878           0.000   

año                       
categoria     vct    vol  
3          20.558 -2.907  
4          18.080 -0.899  
7          15.646 -0.998  
0          16.831  0.252  
8          20.095  0.853  
6          15.307 -1.293  
5          12.593 -0.202  
1          19.019  2.991  
11          6.457 -1.154  
10          0.000  0.000  
9           0.000  0.000  
2           0.000  0.000